In [1]:
import numpy as np, pandas as pd, pystan as ps, numpy.random as npr, matplotlib.pyplot as plt, h5py
%matplotlib inline 
from time import time
from pylab import plot, show, legend
from scipy.stats import pearsonr, spearmanr, norm, invgamma

In [3]:
sm_ng = ps.StanModel(file="ng_logistic.stan") 

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ec2d62b169a03cfdb12340a1ec25c195 NOW.


In [4]:
data = h5py.File("GZZ_data.jld", "r")
X = data["X"].value
y = data["y"].value
ξ_true = data["xi_true"].value
d, Nobs = np.shape(X.transpose())

data = dict(N=Nobs, d=d, y=y.astype(int), X=X)

In [ ]:
start = time()
fit_ng = sm_ng.sampling(data=data, thin=1, n_jobs=4, init="random", iter=5_000, algorithm="HMC")
print(round((time()-start)/60, 2), "mins to run")
print(fit_ng);

In [ ]:
trace = fit_ng.extract()
xi_samples = trace["xi"]

In [ ]:
np.shape(xi_samples)

In [ ]:
cover = np.zeros(d)
ci = np.zeros((d,2))
for i in range(d) :
    ci[i,:] = np.percentile(xi_samples[:,i], q=[5, 95])
    cover[i] = (ci[i,0]<ξ_true[i])&(ξ_true[i]<ci[i,1])

In [ ]:
100*np.mean(cover)

In [ ]:
plt.plot(ci[:,0])
plt.plot(ci[:,1])